In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
from pathlib import Path

In [0]:
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)
path_to_file = Path(path_to_zip).parent.as_posix()+"/spa-eng/spa.txt"
print(path_to_zip)
print(path_to_file)

2646016/2638744 [==============================] - 0s 0us/step
/root/.keras/datasets/spa-eng.zip
/root/.keras/datasets/spa-eng/spa.txt


In [0]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [0]:
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [0]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [0]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [0]:
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
num_examples = 30000

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
max_length_tar, max_leng_inp = target_tensor.shape[1], input_tensor.shape[1]

In [0]:
max_length_tar, max_leng_inp

(11, 16)

In [0]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [0]:
input_tensor_train.shape, input_tensor_val.shape, target_tensor_train.shape, target_tensor_val.shape

((24000, 16), (6000, 16), (24000, 11), (6000, 11))

In [0]:
input_tensor_train[10000]

array([   1,   30,   16, 1741,    3,    2,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [0]:
target_tensor_train[1000]

array([  1,   4,  47,  15, 125,  20,  44,   3,   2,   0,   0], dtype=int32)

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print(f'{t} ----> {lang.index_word[t]}')


In [0]:
convert(inp_lang, input_tensor_train[0])

1 ----> <start>
8256 ----> pesque
15 ----> un
4618 ----> resfrio
3 ----> .
2 ----> <end>


In [0]:
convert(targ_lang, target_tensor_train[0])

1 ----> <start>
4 ----> i
29 ----> have
503 ----> caught
9 ----> a
183 ----> cold
3 ----> .
2 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                     return_sequences=True,
                     return_state=True,
                     recurrent_initializer='glorot_uniform')
    
  def call(self, x, hidden):
    print(x.shape)
    x = self.embedding(x)
    print(x.shape)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [0]:
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_state = encoder(example_input_batch, sample_hidden)

(64, 16)
(64, 16, 256)


In [0]:
sample_output.shape, sample_state.shape

(TensorShape([64, 16, 1024]), TensorShape([64, 1024]))

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    w1 = self.W1(query_with_time_axis)
    w2 = self.W2(values)
    score = self.V(tf.nn.tanh(w1+w2))
    print(f"w1:{w1.shape}, w2:{w2.shape}, score:{score.shape}")

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    # print(f'context_vector:{context_vector.shape}, attention_weights:{attention_weights.shape}')

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

w1:(64, 1, 10), w2:(64, 16, 10), score:(64, 16, 1)


In [0]:
attention_result.shape, attention_weights.shape


(TensorShape([64, 1024]), TensorShape([64, 16, 1]))

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                    return_sequences=True,
                    return_state=True,
                    recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    # print(f'x:{x.shape}')
    x = self.embedding(x)
    # print(f'x:{x.shape}')
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, state, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                     sample_hidden, sample_output)

w1:(64, 1, 1024), w2:(64, 16, 1024), score:(64, 16, 1)


In [0]:
sample_decoder_output.shape, state.shape

(TensorShape([64, 4935]), TensorShape([64, 1024]))

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                              maxlen=max_length_inp,
                              padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,dec_hidden,enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [0]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

In [0]:
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

In [0]:
t1 = tf.constant([[1, 2, 3], [4, 5, 6]])
t2 = tf.constant([[7, 8, 9], [10, 11, 12]])

In [0]:
t3 = []
t3.append(t1)
t3.append(t2)
t3

[<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  8,  9],
        [10, 11, 12]], dtype=int32)>]

In [0]:
tf.stack(t3, axis=1)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 7,  8,  9]],

       [[ 4,  5,  6],
        [10, 11, 12]]], dtype=int32)>

In [0]:
tf.concat([t1, t2], 0)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [0]:
tf.concat([t1,t2], axis=1)

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]], dtype=int32)>

In [0]:
tf.stack([t1, t2], 1)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 7,  8,  9]],

       [[ 4,  5,  6],
        [10, 11, 12]]], dtype=int32)>

In [0]:
tf.stack([t1, t2], 0)

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]]], dtype=int32)>

In [0]:
s = list('abcdefg')
s

In [0]:
z = tf.zeros((2, 4 ,3))
z

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)>

In [0]:
z*2

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [0]:
y = tf.ones((2, 1, 3))
y

<tf.Tensor: shape=(2, 1, 3), dtype=float32, numpy=
array([[[1., 1., 1.]],

       [[1., 1., 1.]]], dtype=float32)>

In [0]:
y+z

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [0]:
x = tf.ones((2, 6))
x

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [0]:
tf.split(x, 3, 1)

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 1.],
        [1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 1.],
        [1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 1.],
        [1., 1.]], dtype=float32)>]

In [0]:
x = tf.ones((3, 4, 1))
x

<tf.Tensor: shape=(3, 4, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        [1.]]], dtype=float32)>

In [0]:
tf.squeeze(x, axis=-1)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [0]:
r = tf.random.uniform((2, 4), 0, 5, dtype=tf.int32)
r

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[3, 1, 4, 0],
       [2, 0, 1, 2]], dtype=int32)>

In [0]:
mask = tf.math.not_equal(r, 0)
mask

<tf.Tensor: shape=(2, 4), dtype=bool, numpy=
array([[ True,  True,  True, False],
       [ True, False,  True,  True]])>

In [0]:
tf.math.logical_not(tf.math.equal(r, 1))

<tf.Tensor: shape=(2, 4), dtype=bool, numpy=
array([[ True, False,  True,  True],
       [ True,  True,  True,  True]])>

In [0]:
 loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

In [0]:
pred = tf.random.uniform((2, 4, 5), 1, 11)
pred

<tf.Tensor: shape=(2, 4, 5), dtype=float32, numpy=
array([[[ 4.546525 ,  7.0668683,  3.5332367,  7.9207993,  2.0179734],
        [ 2.920191 ,  3.4761152,  2.0355306,  2.504444 ,  3.4042368],
        [ 5.7143245,  2.4327931,  9.127663 ,  9.659123 ,  1.5683303],
        [ 5.704009 , 10.206129 ,  3.4108207,  5.3199406,  3.6759624]],

       [[ 4.718035 ,  6.646247 ,  6.9001293,  9.944088 ,  5.274087 ],
        [ 3.9275513,  1.5486155,  5.982773 ,  8.544718 ,  6.395031 ],
        [ 2.4536953,  3.505684 ,  5.6778727,  9.827198 ,  4.9750824],
        [ 9.111958 ,  7.7148724,  2.620655 ,  1.0993943,  8.126879 ]]],
      dtype=float32)>

In [0]:
loss = loss_object(r, pred)
loss 

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1.609438, 1.609438, 1.609438, 1.609438],
       [1.609438, 1.609438, 1.609438, 1.609438]], dtype=float32)>

In [0]:
mask = tf.cast(mask, dtype=loss.dtype)
mask

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1., 1., 1., 0.],
       [1., 0., 1., 1.]], dtype=float32)>

In [0]:
loss *= mask
loss

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[1.609438, 1.609438, 1.609438, 0.      ],
       [1.609438, 0.      , 1.609438, 1.609438]], dtype=float32)>

In [0]:
loss = tf.reduce_sum(loss, axis=1)
loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([4.828314, 4.828314], dtype=float32)>

In [0]:
loss = tf.reduce_mean(loss)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=4.828314>

In [0]:
tf.math.pow(0.9, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.80999994>

In [40]:
import os
params = {}
params['seq2seq_model_dir'] = os.getcwd()
ckpt_dir = f"{params['seq2seq_model_dir']}/checkpoint"
ckpt_dir

'/content/checkpoint'

In [0]:
from pathlib import Path

In [63]:
Path.joinpath(Path('/content'), 'check').as_posix()

'/content/check'

In [64]:
os.path.join('./content', 'check')

'./content/check'

In [21]:
pred = tf.random.uniform((2, 5))
pred

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.16849649, 0.44436777, 0.75113785, 0.7002121 , 0.43558943],
       [0.90965855, 0.13867927, 0.69985247, 0.3770734 , 0.2844473 ]],
      dtype=float32)>

In [22]:
tf.argmax(pred, axis=1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 0])>

In [23]:
tf.constant([1]*2)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 1], dtype=int32)>

In [6]:
d = {w:np.random.uniform(-0.25, 0.25, 3) for i, w in enumerate(list('efvbjop'))}
d

{'b': array([0.06260202, 0.22069447, 0.01653547]),
 'e': array([-0.12364729,  0.04595097, -0.12194141]),
 'f': array([-0.01138323,  0.24460812, -0.14612364]),
 'j': array([-0.09697513, -0.20304291, -0.09355447]),
 'o': array([ 0.06075304, -0.14457917, -0.21196056]),
 'p': array([-0.14584778, -0.12968326, -0.24977316]),
 'v': array([ 0.11337407, -0.04806649, -0.11514531])}

In [7]:
d.keys()

dict_keys(['e', 'f', 'v', 'b', 'j', 'o', 'p'])

In [13]:
d.values()

AttributeError: ignored

In [16]:
len(next(iter(d.values())))

3